# NEW psth plots

Here you just need to identify the suite2p output folder.
Don't forget to add the NPZ file to this folder too.  
Make sure the slashes are /, not backslashes.  
Confirm image sampling rate.

Get cell traces that are identified as cells by suite2p  
Get metadata

In [ ]:
### FOR TESTING ONLY #######

test_mode = False
user = "carmen"

if test_mode and user == 'carmen':
    output_subfolder = 'C:/Users/huynh/temp_local/carmen_feb_sce/carmen_pmth_env'
    output_folder = 'C:/Users/huynh/temp_local/carmen_feb_sce/carmen_pmth_env/output_test'
    suite2p_folder = 'C:/Users/huynh/temp_local/carmen_feb_sce/carmen_pmth_env/suite2p'
    file_name = 'test_name' 
    main_excel_path = 'C:/Users/huynh/temp_local/carmen_feb_sce/carmen_pmth_env/pmth_main.xlsx'

## TEST 2: fixing detrending
if test_mode and user == "phan":
    print('testing mode, user phan')
    output_subfolder = 'C:/Users/huynh/temp_local/carmen_feb_sce/carmen_pmth_env'
    output_folder = 'C:/Users/huynh/temp_local/carmen_feb_sce/carmen_pmth_env/output_test'
    suite2p_folder = 'C:/Users/huynh/temp_local/carmen_feb_sce/carmen_pmth_env/suite2p_15july'
    file_name = 'test_name' 
    main_excel_path = 'C:/Users/huynh/temp_local/carmen_feb_sce/carmen_pmth_env/pmth_main.xlsx'
    excel_row_id = 0

In [ ]:
# output_folder = {output_subfolders}
# suite2p_folder = {suite2p_paths} 
imaging_sampling_rate = 10.31476027 # For carmen (2 planes on Bruker), basically same as Caro's (10.31475985). taken from XML


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import os
import pandas as pd

F = np.load(os.path.join(suite2p_folder, 'F.npy'))
iscell = np.load(os.path.join(suite2p_folder, 'iscell.npy'))

print('n_cells before suite2p filter (from iscell): ', F.shape[1])

traces = F[iscell[:,0]==1] # Confirmed in Cicada
n_cells, n_frames = traces.shape
seconds_per_frame = 1 / imaging_sampling_rate
frames_per_second = imaging_sampling_rate
print("n_cells chosen by suite2p: ", n_cells)
print("n_frames: ", n_frames)
print("seconds: ", n_frames/imaging_sampling_rate)
print("minutes: ", n_frames/imaging_sampling_rate/60)

Define some preprocessing functions and apply to traces    
This is based on JC's preprocessing for assembly detection 

In [ ]:
# Import libraries
import matplotlib as plt
import scipy
from scipy import signal
from scipy.stats import iqr
from scipy.signal import find_peaks
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt


# Filters from JC's Matlab version

def median_normalization(traces):
    n_cells, n_frames = traces.shape
    for i in range(n_cells):
        traces[i, :] = traces[i, :] / np.median(traces[i, :])
    return traces

# def bleaching_correction(traces): ### JC's VERSION, but doesn't work as well as original CICADA version below 
#     n_cells, n_frames = traces.shape
#     for k in range(n_cells):
#         p0 = np.polyfit(np.arange(n_frames), traces[k, :], 3) ## originally 3
#         traces[k, :] = (traces[k, :] / np.polyval(p0, np.arange(n_frames)))-np.mean(traces[k,:]) # PHAN: replaced this poly with detrend to better match JC's Matlab version.
#     return traces

def bleaching_correction(traces):  ### CICADA version, seems to work better than JC's matlab version.
    n_cells, n_frames = traces.shape
    for k in range(n_cells):
        p0 = np.polyfit(np.arange(n_frames), traces[k, :], 3)
        traces[k, :] = traces[k, :] / np.polyval(p0, np.arange(n_frames))
    return traces

def savitzky_golay_filt(traces): ### 
    traces = signal.savgol_filter(traces, 7, 3, axis=1)  #PHAN:  changed framelength from 5 to 7 to match JC's Matlab version.
    return traces



In [ ]:
traces = bleaching_correction(traces).copy()

In [ ]:
# # Filters from JC's Matlab version
# traces_for_sce = median_normalization(traces).copy()
# traces_for_sce = bleaching_correction(traces_for_sce).copy()
# traces_for_sce = savitzky_golay_filt(traces_for_sce).copy()


In [ ]:
# ### CHECKING Bleaching correction:

# array1 = traces_dff
# array_copy = traces.copy()
# array2 = bleaching_correction(array_copy)

# n_random = 10
# import numpy as np
# import plotly.graph_objects as go
# import random

# ### GOOD CELL TO TEST: 145
# selected_indices = random.sample(range(array1.shape[0]), n_random)
# selected_indices = [145]

# print(array1[selected_indices][:5])
# print()
# print(array2[selected_indices][:5])

# fig = go.Figure()

# # Add traces for the selected time series from array1
# for idx in selected_indices:
#     fig.add_trace(go.Scatter(y=array1[idx], mode='lines', name=f'DF/F', line=dict(color='blue', width=1)))

# # Add traces for the selected time series from array2
# for idx in selected_indices:
#     fig.add_trace(go.Scatter(y=array2[idx]*100, mode='lines', name=f'Detrended', line=dict(color='red', width=1)))


# # Update layout
# fig.update_layout(
#     title=f'Visualizing detrending vs dff normalization, cell {selected_indices}',
#     xaxis_title='Time Points',
#     yaxis_title='Activity',
#     legend_title='Series',
#     template='plotly_white'
# )

# # Show plot
# fig.show()


## Detect when cells are active/spikes (SLOW)
Definition of active taken from JC.     
This step takes some time to process.  
To save time, I've exported the output (saved in same suite2p folder) to be retrieved later.   
To comment this box out (so it doesn't execute), highlight all the text (ctrl + a) and then ctrl + /.

In [ ]:
# # Detect Spikes traces in each trace, using JC's max of 3 threshold

traces_for_sce = traces 

n_cells, n_frames = traces_for_sce.shape

window_size = int(4 * imaging_sampling_rate)   # 4 seconds long window as Arnaud was doing with 40 in 10Hz recording
print(f"Window size is {window_size} frames")

minithreshold = 0.2
MinPeakDistance = 3

WinActive = [] # WinActive = np.where(speed > 1)[0]

th_cic = []
activity_tmp_all_cells = [[] for i in range(n_cells)]

if test_mode:
    activity_tmp_all_cells_array = np.load(os.path.join(output_folder, 'activity_tmp_all_cells.npy'), allow_pickle=True)
    activity_tmp_all_cells = list(activity_tmp_all_cells_array)
else:

    for i in range(n_cells):
        activity_tmp_all_cells = [None] * n_cells
        for i in range(n_cells):
            th_i = max([3 * iqr(traces_for_sce[i]), 3 * np.std(traces_for_sce[i]), minithreshold])
            th_cic.append(th_i)
            peaks, properties = find_peaks(traces_for_sce[i], prominence=th_i, distance=MinPeakDistance)
            valeurs_identiques = np.intersect1d(peaks, WinActive)
            locs_sans_ide = np.setdiff1d(peaks, valeurs_identiques)
            activity_tmp_all_cells[i] = locs_sans_ide

    #convert list of arrays to array of arrays in order to save to npy.file
    activity_tmp_all_cells_array = np.array(activity_tmp_all_cells, dtype=object)

    np.save(os.path.join(output_folder, 'activity_tmp_all_cells.npy'), activity_tmp_all_cells_array)



If output is already saved, here you can retrieve.

In [ ]:
n_spikes = sum(len(arr) for arr in activity_tmp_all_cells)
spikes_per_sec = n_spikes/(n_frames/imaging_sampling_rate)
spikes_per_sec_per_cell = spikes_per_sec/n_cells

# print("n_spikes:", n_spikes)
# print("spikes_per_min_per_cell:", spikes_per_min_per_cell)

Here we find timepoints where there are multiple active cells (over 10).  
This cell's output is the main info you're looking for.  The rest are just plots.

In [ ]:
def convert_to_raster(activity_tmp_all_cells):
    raster = np.zeros((n_cells, n_frames))
    for i in range(n_cells):
        raster[i, activity_tmp_all_cells[i]] = 1
    return raster

def get_sce_threshold(rasterdur, n_surrogates=100, percentile=99, verbose=False): # ROBIN's Paper:  300 n_surr, 99th percentile.  Cicada default is 100, 95.  Changed cicada default per Rosa (5 June meeting)
    [n_cells, n_frames] = rasterdur.shape
    if verbose:
        print(f"Starting to obtain the {n_surrogates} rolled rasters")
    rnd_rasters = np.zeros((n_cells, n_frames, n_surrogates))
    for surrogate in range(n_surrogates):
        for cell in range(n_cells):
            rnd_rasters[cell, :, surrogate] = np.roll(rasterdur[cell, :], np.random.randint(1, n_frames))

    rnd_sum_cells = np.sum(rnd_rasters, axis=0)
    n_values = n_frames * n_surrogates
    rnd_sum_cells_vector = np.reshape(rnd_sum_cells, (n_values,))
    cell_threshold = np.percentile(rnd_sum_cells_vector, percentile)

    if verbose:
        print(f"Threshold obtain for SCE detection with {percentile} percentile: {cell_threshold} cells, "
              f"representing {np.round((cell_threshold / n_cells) * 100, decimals=1)} % of the detected cells")

    return rnd_rasters, cell_threshold

In [ ]:
from scipy import signal, stats

matrix, sce_n_cells_threshold = get_sce_threshold(convert_to_raster(activity_tmp_all_cells))
sce_manual_threshold = 5 
sce_percent_threshold = int(.03*n_cells)
sce_min_distance_sec = 1

sce_min_distance_frames = sce_min_distance_sec*frames_per_second # before, it was 4 (default from CICADA), but changed to 3 (10 june) based on JC's assembly_SCE6.m on github

print("95% percentile of surrogate: ", sce_n_cells_threshold)  ###  
print("3% of cells: ", sce_percent_threshold)  ### lowering to 3% based on Carmen convo (10 JUNE)
print("manual minimum: 5")
n_synchronous_frames = 2

# sce_n_cells_threshold = max(sce_calc_threshold, sce_percent_threshold,sce_manual_threshold) ### 5% of cells removed per Rosa (April 25)
# sce_n_cells_threshold = sce_calc_threshold.copy() ### USE ONLY sce_calc_threhold, at 99% and 1 sec min distance, per rosa 12 june

print("sce_n_cells_threshold: ", sce_n_cells_threshold)


raster = np.zeros((n_cells, n_frames))

for i in range(n_cells):
    raster[i, activity_tmp_all_cells[i]] = 1

# sum activity over n consecutive frames
sum_activity = np.zeros(n_frames - n_synchronous_frames)
for i in range(n_frames - n_synchronous_frames):
    sum_activity[i] = np.sum(np.amax(raster[:, np.arange(i, i + n_synchronous_frames)], axis=1))

# select synchronous calcium events
sce_loc = signal.find_peaks(sum_activity, height=sce_n_cells_threshold, distance=sce_min_distance_frames)[0]
n_sce = len(sce_loc)
sce_trace = np.array([np.array([1 if i in sce_loc else 0 for i in range(n_frames)])])

sce_per_sec = n_sce/(n_frames/imaging_sampling_rate)
print("SCEs detected: ", n_sce)
print("SECs per second: ", n_sce/(n_frames/imaging_sampling_rate), "(range should be .1 to .5)")
print("SECs per minute: ", n_sce/(n_frames/imaging_sampling_rate/60))


### comparing n_sce to 3% of cells:
# select synchronous calcium events
sce_loc_percent = signal.find_peaks(sum_activity, height=sce_percent_threshold, distance=sce_min_distance_frames)[0]
n_sce_percent = len(sce_loc_percent)
print("n_sce_percent: ", n_sce_percent)
sce_loc_manual = signal.find_peaks(sum_activity, height=sce_manual_threshold, distance=sce_min_distance_frames)[0]
n_sce_manual = len(sce_loc_manual)
print("n_sce_manual: ", n_sce_manual)

### create cells vs sce matrix
# sce_cells_matrix = np.zeros((n_cells, n_sce))
# for i in range(n_sce):
#     sce_cells_matrix[:, i] = np.amax(raster[:, np.arange(np.max((sce_loc[i]-1, 0)),
#                                                             np.min((sce_loc[i]+2, n_frames)))], axis=1)


Before plotting, I use TSNE to reorder the cells.  
(I don't think it helps much)

In [ ]:
# from sklearn.manifold import TSNE

# #### APPLY TSNE to reorder sce matrix

# # Assuming Tr1b is your data, replace it with your actual data
# Tr1b = sce_cells_matrix.copy()

# # Set TSNE parameters
# tsne = TSNE(
#     n_components=1, # how many dimensions to reduce to, 1 is just a list, 2 for 2D, 3 for 3D, etc.
#     # n_iter=500,
#     # perplexity=1, #
#     # early_exaggeration=500,
#     # metric="correlation",
#     # method="barnes_hut",
#     random_state=42  # Set a random state for reproducibility
# )

# # Fit TSNE and get the transformed data
# TSNE_result = tsne.fit_transform(Tr1b)

# # Get sorted indices
# index = np.argsort(TSNE_result[:, 0])

# # Sort the TSNE result based on the sorted indices
# B = TSNE_result[index]
# # Now B and index contain the equivalent results to the MATLAB code


In [ ]:
# from sklearn.cluster import DBSCAN

# dbscan = DBSCAN()

# tsne = TSNE(
#     n_components=3, # how many dimensions to reduce to, 1 is just a list, 2 for 2D, 3 for 3D, etc.
#     # n_iter=500,
#     # perplexity=1, #
#     # early_exaggeration=500,
#     # metric="correlation",
#     # method="barnes_hut",
#     random_state=42  # Set a random state for reproducibility
# )

# # Fit TSNE and get the transformed data
# TSNE_result = tsne.fit_transform(Tr1b)
# labels = dbscan.fit_predict(TSNE_result)

Make raster plot (with and without TSNE cell ordering).  
This plot shows just the timepoints where there are at least ten cells active.

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns

# # Create a figure and set the size
# plt.figure(figsize=(20, 10))

# # Define the number of rows and columns for the subplots
# rows = 3
# cols = 1

# # Plot the first heatmap (top one)
# plt.subplot(rows, cols, 1)
# sns.heatmap(sce_cells_matrix, cmap='gray_r', cbar=True)

# # Add title and axis labels for the top heatmap
# plt.title('SCE cells matrix (unsorted)')
# plt.xlabel('SCE (frame)')
# plt.ylabel('Cell ID')

# # Plot the second heatmap (bottom one)
# plt.subplot(rows, cols, 2)
# sns.heatmap(sce_cells_matrix[index], cmap='gray_r', cbar=True)

# plt.subplot(rows, cols, 3)
# sns.heatmap(sce_cells_matrix[index1], cmap='gray_r', cbar=True)

# # Add title and axis labels for the bottom heatmap
# plt.title('SCE cells matrix (sorted)')
# plt.xlabel('SCE (frame)')
# plt.ylabel('Cell ID')

# # Adjust layout to prevent overlap
# plt.tight_layout()

# # Show the plot
# plt.show()


## Create heatmap of all cell activities across all timepoints

Get behavior annotation from NPZ file

In [ ]:
# from numpy import load
# import os
# import numpy as np 
# suite2p_folder = 'D:/phan/Plane0/suite2p_231011_231023_Plane0'

In [ ]:
## GET EPOCH TIMES FROM NPZ #### (In cicada, it takes in timestamps in seconds (probably from NPZ) and converts to frames 
from numpy import load


files = os.listdir(suite2p_folder)
npz_file = [file for file in files if file.endswith('.npz')][0]
npz_data = load(os.path.join(suite2p_folder, npz_file),allow_pickle=True)

data_types = npz_data.files #['Twitches', 'Startles', 'Complex_Movements']
print("data_types before: ", data_types)
twitch_string = next(s for s in data_types if 'twitch' in s.lower())
startle_string = next(s for s in data_types if 'start' in s.lower())
complex_string = next(s for s in data_types if 'complex' in s.lower())

movement_types_all = [twitch_string,startle_string,complex_string ] # remove MotCorr Auto Detection
movement_types_no_complex = [twitch_string, startle_string]
n_movement_types = len(movement_types_all)

twitches = (npz_data[twitch_string][0]*imaging_sampling_rate).astype(int)
startles = (npz_data[startle_string][0]*imaging_sampling_rate).astype(int)
complex = (npz_data[complex_string][0]*imaging_sampling_rate).astype(int)

movements_frame_list = [twitches, startles, complex]

lst = movement_types_all # movement_types_no_complex
# for item in lst:
#     print(item)
#     print(data[item][0])
#     # print((data[item][0]*imaging_sampling_rate).astype(int))
#     # movements = np.concatenate((movements, data[item][0]))
# print("")

movements_frame = []
movements_sec = []
for item in lst:
    sec_temp = npz_data[item][0]
    movements_sec.extend(sec_temp)
    
    frame_temp = (npz_data[item][0]*imaging_sampling_rate).astype(int)
    movements_frame.extend(frame_temp)
    # movement_frames = np.concatenate(movement_frames, frame_temp)

#Convert to frames
movements_sec = np.array(movements_sec)
movements_frame = np.array(movements_frame, dtype=np.int32)

# movements = np.sort(movements)
print("")

movement_count = len(movements_frame)
movement_per_sec = movement_count/(n_frames/imaging_sampling_rate)
movement_per_min = movement_count/(n_frames/imaging_sampling_rate/60)
print("count: ", len(movements_frame))
print("movement_per_sec: ", movement_per_sec)
print("movement_per_min: ", movement_per_min)
# print(movements_sec)
# print(np.sort(movements_frame))

fake_movements_frame = np.random.randint(0, n_frames+1, len(movements_frame))

In [ ]:
def match_sce_to_movement(sce, movement, buffer):
    sce_matched = []
    sce_idx_matched = []
    for i, a in enumerate(sce):
        for j, b in enumerate(movement):
            if abs(a - b) <= buffer:
                if a not in sce_matched:
                    sce_matched.append(a)
                    # sce_idx_matched.append(i)
                break  # Move to the next element in A
    return np.array(sce_matched)


You can adjust the brightness of the heatmap by changing the max_bright and min_bright values

In [ ]:
# Try different normalization methods to make rasterplot more visible

from scipy import stats
traces_zscore = stats.zscore(traces, axis=1)

# min max normalize traces at each row
traces_min = np.min(traces, axis=1)
traces_max = np.max(traces, axis=1)
traces_minmax = (traces - traces_min[:, np.newaxis]) / (traces_max - traces_min)[:, np.newaxis]

To save the interactive heatmap in your suite2p folder, set "save_to_html" = True.

In [ ]:
import plotly.graph_objects as go
from scipy.stats import zscore

export_heatmap_sce = True # check folder path 
plot_traces = traces_zscore # traces_zscore # traces_minmax # traces_zscore
order = range(0,n_cells) # index # isort # tsne_index

# plot trace average.  
traces_sum = np.sum(traces, axis=0)
traces_avg = traces_sum / n_cells
traces_line = traces_avg # for plotting
traces_line = (traces_line - np.min(traces_line))*1
# Assuming traces_minmax, index, movements, imaging_sampling_rate, traces, and sce_loc are your data arrays

# Calculate the maximum amplitude for sum_activity and line_trace
max_amplitude = 0.75 * n_cells
# max_sum_activity = np.max(sum_activity_full) * 10
max_traces_line = np.max(traces_line)
# sum_activity_scaled = (sum_activity_full / max_sum_activity) * max_amplitude * magnification_factor
traces_line_scaled = (traces_line / max_traces_line) * max_amplitude 

sce_matched = match_sce_to_movement(sce_loc, movements_frame, 5)
sce_not_matched = np.setdiff1d(sce_loc, sce_matched)
# print(sce_matched)

# Create a heatmap
heatmap_trace = go.Heatmap(
    z=plot_traces[order],
    # z = zscore(traces[order], axis=1)
    colorscale= 'Reds', # gray_r = reverse color
    showscale=True,
    name='Heatmap',  # Name for the legend
    zmin=0,  # zmin, Set a suitable minimum value for the colormap
    zmax=1.96, # 1.96 is one std, what you want for zscore
    colorbar=dict(len=0.4)  # Set the length of the color scale to 40% of the original length
)

# Mark SCEs
sce_matched = go.Scatter(
    x=sce_matched,
    y=[n_cells - 1] * n_frames,  
    # y=[0] * len(sce_loc),
    mode='markers',
    marker=dict(color='green', symbol='triangle-down', size=10),
    name=f'{len(sce_matched)} matched SCEs',  # Name for the legend
)

# Mark SCEs
sce_not_matched = go.Scatter(
    x=sce_not_matched,
    y=[n_cells - 1] * n_frames,  
    # y=[0] * len(sce_loc),
    mode='markers',
    marker=dict(color='red', symbol='triangle-down', size=10),
    name=f'{len(sce_not_matched)} unmatched SCEs',  # Name for the legend
)

# Mark behavior
scatter_twitches = go.Scatter(
    x=twitches,
    y=[0] * len(movements_frame),
    mode='markers',
    marker=dict(color='red', symbol='triangle-up', size=10),
    name=f'{len(twitches)} twitches',  # Name for the legend
)

# Mark startles
scatter_startles = go.Scatter(
    x=startles,
    y=[0] * len(movements_frame),
    mode='markers',
    marker=dict(color='blue', symbol='triangle-up', size=10),
    name=f'{len(startles)} startles',  # Name for the legend
)

# Mark complex movements
scatter_complex = go.Scatter(
    x=complex,
    y=[0] * len(movements_frame),
    mode='markers',
    marker=dict(color='green', symbol='triangle-up', size=10),
    name=f'{len(complex)} complex',  # Name for the legend
)

# # Draw line of average 
# line_trace = go.Scatter(
#     x=np.arange(len(traces_line)),
#     y=traces_line_scaled,
#     mode='lines',
#     line=dict(color='red', width=.5),  # Set line color and width
#     name='traces_avg',  # Name for the legend
# )

# # Draw line of spike count 
# sum_activity = go.Scatter(
#     x=np.arange(len(traces_line)),
#     y=sum_activity_full*10,
#     mode='lines',
#     line=dict(color='red', width=2),  # Set line color and width
#     name='sum_activity',  # Name for the legend
# )

# Create layout with aspect ratio adjustment
layout = go.Layout(
    title=(f'Cell activity with SCE and behavior annotations<br>{file_name}'),
    xaxis=dict(title='Time (frame)'),
    yaxis=dict(title='Cell ID'),
    height=700,  # Set the height of the plot
    width=1300,  # Set the width of the plot
)

# Create figure with adjusted legend order
# fig = go.Figure(data=[heatmap_trace, sce_marker, scatter_twitches, scatter_startles, scatter_complex], layout=layout)

fig = go.Figure(data=[heatmap_trace, sce_matched, sce_not_matched, scatter_twitches, scatter_startles, scatter_complex], layout=layout)

# Show the plot
fig.show()

if export_heatmap_sce:
    fig.write_html(os.path.join(output_folder, "heatmap_sce.html"))

## Create PSTH plots

In [ ]:
def convert_to_dff (traces):
    dff = []
    for trace in traces:
        dff_trace = (trace - np.median(trace))*100 / np.median(trace)
        dff.append(np.array(dff_trace))
    return np.vstack(dff)



In [ ]:
###### DEBLEACHING IS ALREADY A NORMALIZING, instead, we'll zscore to make values closer and make heatmap more pretty.

traces_dff = convert_to_dff(traces)

In [ ]:
### Calculate MULTIPLE ave spikes from MULTIPLE traces
from scipy.stats import ttest_ind

traces_to_plot = traces_dff
# traces_to_plot = traces

range_value=104
lower_percent = 5 
upper_percent = 95

post_analysis_length = 50 # from center
pre_analysis_length = 25 # from center 

avg_spike_all_traces = []
trace_length = len(traces[0])  # Assuming all traces have the same length
positive_trace_index = []
neutral_trace_index = []
negative_trace_index = []
max_spike_all_traces = []
min_spike_all_traces = []
margins_to_threshold = []
t_values = []
p_values = []
subsegment_avg = []
subsegment_max = []
subsegment_min = []
pre_avgs = []
pre_stds = []
post_z_max = []
post_z_min = []

for trace in traces_to_plot:
    spikes_per_trace = []

    upper_threshold = np.percentile(traces_to_plot, upper_percent)
    lower_threshold = np.percentile(traces_to_plot, lower_percent)
    
    # upper_threshold = np.median(trace)+np.std(trace)
    # lower_threshold = np.median(trace)-np.std(trace)
    
    # print("upper_threshold:", upper_threshold)
    # print("lower_threshold: ", lower_threshold)
    for frame_index in movements_frame:
        # Calculate the start and end index for the segment
        beg_frame = max(frame_index - range_value, 0)  # Ensure beg_frame is not negative
        end_frame = min(frame_index + range_value + 1, trace_length)  # Ensure end_frame is within trace_length
        
        # Extract the segment from the time series and pad with zeros if necessary
        segment = trace[beg_frame:end_frame]

        if len(segment) < 2 * range_value + 1:
            pad_length = 2 * range_value + 1 - len(segment)
            segment = list(segment)
            segment.extend([0] * pad_length)
            segment = np.array(segment)          
        
        spikes_per_trace.append(segment)



    # print(len(segments))
    # Compute the average of the segments for the current trace
    avg_spike_per_trace = [sum(values) / len(spikes_per_trace) for values in zip(*spikes_per_trace)]
    
    subsegment_avg.append(np.average(avg_spike_per_trace[range_value:range_value+post_analysis_length]))
    subsegment_max.append(np.max(avg_spike_per_trace[range_value:range_value+post_analysis_length]))
    subsegment_min.append(np.min(avg_spike_per_trace[range_value:range_value+post_analysis_length]))

    # Perform t-test between pre and post movement 

    # pre_avg_spike_per_trace = avg_spike_per_trace[:range_value]
    # post_avg_spike_per_trace = avg_spike_per_trace[range_value+1:]

    # pre_avg_spike_per_trace = avg_spike_per_trace[range_value-pre_analysis_length : range_value]

    pre_avg_spike_per_trace = avg_spike_per_trace[: range_value] ### WHOLE WINDOW (NOT USING pre_analysis_length)
    post_avg_spike_per_trace = avg_spike_per_trace[range_value: range_value+post_analysis_length]

    t_stat, p_value = ttest_ind(post_avg_spike_per_trace, pre_avg_spike_per_trace)
    t_values.append(t_stat)
    p_values.append(p_value)

    max_spike_all_traces.append(np.max(avg_spike_per_trace))                                    
    min_spike_all_traces.append(np.min(avg_spike_per_trace))
    
    margins_to_threshold.append(upper_threshold-np.max(avg_spike_per_trace))

    # print("max of avg_spike_per_trace: ",np.max(avg_spike_per_trace)
    if np.max(avg_spike_per_trace) > upper_threshold:
            positive_trace_index.append(trace)
    if np.min(avg_spike_per_trace) < lower_threshold:
            negative_trace_index.append(trace)
    if np.min(avg_spike_per_trace) >= lower_threshold and np.max(avg_spike_per_trace) <= upper_threshold:
            neutral_trace_index.append(trace)
    
    avg_spike_all_traces.append(avg_spike_per_trace)

    #### APPLYING ARTEM'S Z SCORE METHOD
    pre_avg = np.average(avg_spike_per_trace[range_value-pre_analysis_length : range_value]) ### Pre window is used, NOT WHOLE WINDOW.
    pre_avgs.append(pre_avg)
    pre_std = np.std(avg_spike_per_trace[range_value-pre_analysis_length : range_value]) ### Pre window is used, NOT WHOLE WINDOW.
    pre_stds.append(pre_std)
    # print("post_avg_spike_per_trace: ", type(post_avg_spike_per_trace))
    post_z = (np.array(post_avg_spike_per_trace) - pre_avg)/pre_std
    post_z_max.append(np.max(post_z[:post_analysis_length]))
    post_z_min.append(np.min(post_z[:post_analysis_length]))


In [ ]:
# #### Plotting the histogram
# plt.figure(figsize=(10, 6))
# plt.hist(post_z_max, bins=30, edgecolor='black')
# plt.title('Histogram of post_z_max')
# plt.xlabel('post_z_max values')
# plt.ylabel('Frequency')
# plt.grid(True)
# plt.show()

In [ ]:
# #### Plotting the histogram
# plt.figure(figsize=(10, 6))
# plt.hist(post_z_min, bins=30, edgecolor='black')
# plt.title('Histogram of post_z_min')
# plt.xlabel('post_z_min values')
# plt.ylabel('Frequency')
# plt.grid(True)
# plt.show()

In [ ]:
### PLOTTING ALL TRACES

export_movement_average = True # check folder path 

fig = go.Figure()

x_values = np.linspace(-range_value, range_value, len(avg_spike_all_traces[0]))

# Plot individual traces
for spike in avg_spike_all_traces:
    fig.add_trace(go.Scatter(x=x_values, y=spike, mode='lines', name='Individual Trace', line=dict(color='lightgrey', width=1), showlegend=False))

# Plot mean trace
mean_line = np.mean(avg_spike_all_traces, axis=0)
fig.add_trace(go.Scatter(x=x_values, y=mean_line, mode='lines', name='Mean Trace', line=dict(color='black', width=3), showlegend=False))

# # Add horizontal lines to the figure
# for line in horizontal_lines:
#     fig.add_trace(line)

# Update layout
fig.update_layout(
    title=(f'PMTH traces and average<br>{file_name}'), 
    xaxis_title='Time (frame)', 
    yaxis_title='df/f %', 
    xaxis=dict(range=[-range_value, range_value]),  # Set x-axis range from -104 to 104
    width=600, 
    height=400  # Set the height to 400
)
# Show plot
fig.show()

if export_movement_average:
    fig.write_html(os.path.join(output_folder, "pmth_average.html"))

In [ ]:
import numpy as np

threshold_value = 0.05 # for p values

# Get indices of values above and below 0.05
trace_id_sig = [i for i, p_val in enumerate(p_values) if p_val <= threshold_value]
trace_id_insig = [i for i, p_val in enumerate(p_values) if p_val > threshold_value]

# Sort only certain indices based on the condition
trace_id_sorted_sig = sorted(trace_id_sig, key=lambda idx: t_values[idx], reverse=True)
trace_id_sorted_insig = sorted(trace_id_insig, key=lambda idx: t_values[idx], reverse=True)

trace_id_sorted_sig_pos = []
trace_id_sorted_sig_neg = []

for i, idx in enumerate(trace_id_sorted_sig):
    if t_values[idx] >= 0:
        trace_id_sorted_sig_pos.append(idx)
        # print('pos ', i, ' ', t_values[idx])
    else:
        trace_id_sorted_sig_neg.append(idx)
        # print('neg ',i,' ', t_values[idx])

# Print the sorted indices
# print("Sorted indices sig:", trace_id_sorted_sig)
# print("Sorted indices insig:", trace_id_sorted_insig)

# print(len(trace_id_sorted_sig), ' ', len(trace_id_sorted_insig))
# print(len(trace_id_sorted_sig)+len(trace_id_sorted_insig))
print(len(trace_id_sorted_sig_pos), ' ', len(trace_id_sorted_insig), ' ',len(trace_id_sorted_sig_neg) )
print(len(trace_id_sorted_sig_pos)+len(trace_id_sorted_sig_neg)+len(trace_id_sorted_insig))

######## CARMEN VERSION 2, USING JUST THE AVERAGE OF THE 20 frames after movement ########

# Sort only certain indices based on the condition

# subsegment_order = np.array(subsegment_max)


# trace_id_sorted_sig_2 = list(trace_id_sig[np.argsort(subsegment_order[trace_id_sig])[::-1]])
# trace_id_sorted_insig_2 = list(trace_id_insig[np.argsort(subsegment_order[trace_id_insig])[::-1]])

# trace_id_sorted_sig_pos_2 = []
# trace_id_sorted_sig_neg_2 = []

# for i, idx in enumerate(trace_id_sorted_sig_2):
#     if subsegment_order[idx] >= 0:
#         trace_id_sorted_sig_pos_2.append(idx)
#         # print('pos ', i, ' ', t_values[idx])
#     else:
#         trace_id_sorted_sig_neg_2.append(idx)
#         # print('neg ',i,' ', t_values[idx])

######## CARMEN VERSION 3, AFTER SEPERATING INTO THREE (t pos, t neg, and t insgig), THEN SORTING MAX AMP OF THE 20 frames after movement ########

# subsegment_order = np.array(subsegment_max)

# trace_id_sorted_sig_pos_2 = list(trace_id_sig[np.argsort(subsegment_order[trace_id_sorted_sig_pos])[::-1]])
# trace_id_sorted_sig_neg_2 = list(trace_id_sig[np.argsort(subsegment_order[trace_id_sorted_sig_neg])[::-1]])
# trace_id_sorted_insig_2 = list(trace_id_insig[np.argsort(subsegment_order[trace_id_insig])[::-1]])

######### ARTERM VERSION OF SORTING 
subsegment_order = np.array(subsegment_max)

upper_z_threshold = 1.96 
lower_z_threshold = -1.65

trace_id_sig_pos = np.array([i for i in range(len(post_z_max)) if post_z_max[i] >= upper_z_threshold and post_z_min[i] >= lower_z_threshold])
trace_id_sig_neg = np.array([i for i in range(len(post_z_max)) if post_z_max[i] <= upper_z_threshold and post_z_min[i] <= lower_z_threshold])
trace_id_insig = np.array([i for i in range(len(post_z_max)) if (post_z_max[i] < upper_z_threshold and post_z_min[i] > lower_z_threshold) or (post_z_max[i] > upper_z_threshold and post_z_min[i] < lower_z_threshold)])

if trace_id_sig_neg.size > 0:  # Check if trace_id_sig_neg is not empty
    trace_id_sorted_sig_neg = list(trace_id_sig_neg[np.argsort(subsegment_order[trace_id_sig_neg])[::-1]])
else:
    trace_id_sorted_sig_neg = []  # Assign an empty list if trace_id_sig_neg is empty

if trace_id_sig_pos.size > 0:  # Check if trace_id_sig_pos is not empty
    trace_id_sorted_sig_pos = list(trace_id_sig_pos[np.argsort(subsegment_order[trace_id_sig_pos])[::-1]])
else:
    trace_id_sorted_sig_pos = []  # Assign an empty list if trace_id_sig_pos is empty

if trace_id_insig.size > 0:  # Check if trace_id_insig is not empty
    trace_id_sorted_insig = list(trace_id_insig[np.argsort(subsegment_order[trace_id_insig])[::-1]])
else:
    trace_id_sorted_insig = []  # Assign an empty list if trace_id_insig is empty


print(len(trace_id_sorted_sig_pos), ' ', len(trace_id_sorted_sig_neg), ' ',len(trace_id_sorted_insig) )
print(len(trace_id_sorted_sig_pos)+len(trace_id_sorted_sig_neg)+len(trace_id_sorted_insig))

In [ ]:
# ### HISTOGRAM OF P VALUES
# histogram = go.Histogram(
#     x=p_values,
#     nbinsx=50,  # Number of bins
#     marker=dict(color='blue'),  # Color of the bars
#     opacity=0.75  # Opacity of the bars
# )
# layout = go.Layout(
#     title='Histogram of p-values',
#     xaxis=dict(title='p-value'),
#     yaxis=dict(title='Frequency'),
#     height=750,
#     width=750,
#     shapes=[
#         # Vertical line at the threshold value
#         dict(
#             type="line",
#             xref="x",
#             yref="paper",
#             x0=threshold_value,
#             y0=0,
#             x1=threshold_value,
#             y1=1,
#             line=dict(color="red", width=2)
#         )
#     ]
# )

# # Create the figure
# fig = go.Figure(data=[histogram], layout=layout)

# # Show the figure
# fig.show()

In [ ]:
import plotly.graph_objs as go

export_movement_heatmap = True

sig_pos_count = len(trace_id_sorted_sig_pos)
sig_neg_count = len(trace_id_sorted_sig_neg)
insig_count = len(trace_id_sorted_insig)

total_count = sig_pos_count + sig_neg_count + insig_count


# plot_traces = np.array(avg_spike_all_traces)
plot_traces = stats.zscore(avg_spike_all_traces, axis=1)

order = trace_id_sorted_sig_pos + trace_id_sorted_insig + trace_id_sorted_sig_neg


order = order[::-1]
line1 = len(trace_id_sorted_sig_neg)
line2 = len(trace_id_sorted_sig_neg)+len(trace_id_sorted_insig)
first_frame = int((len(plot_traces[0])-1)/-2)
x_values = list(range(first_frame, first_frame + len(plot_traces)))


custom_colorscale = [
    [0.0, 'blue'],    # Start with white
    [0.5, 'white'],
    [1.0, 'red']     # End with black
]

layout = go.Layout(
    title=(f'Sorted PMTH heatmap <br>{file_name} <br>'
           f'pos. t: {int(100*sig_pos_count/total_count)}% ({sig_pos_count}/{total_count})   |   '
           f'insig. t: {int(100*insig_count/total_count)}% ({insig_count}/{total_count})   |   '
           f'neg. t: {int(100*sig_neg_count/total_count)}% ({sig_neg_count}/{total_count})'),        
    xaxis=dict(title='Time (frame)'),
    yaxis=dict(title='Cells (middle group: p_val > 0.05)'),
    height=650,
    width=750,
    shapes=[
        dict(type="line", xref="paper", yref="y", x0=0, y0=line1, x1=1, y1=line1, line=dict(color="black", width=2)),
        dict(type="line", xref="paper", yref="y", x0=0, y0=line2, x1=1, y1=line2, line=dict(color="black", width=2))
    ]
)

fig = go.Figure(layout=layout)


# Add the heatmap trace with zmin and zmax set to -2 and 2
fig.add_trace(go.Heatmap(
    x=x_values, 
    z=plot_traces[order], 
    colorscale=custom_colorscale,
    zmin=-1.96,
    zmax=1.96
))
fig.show()

if export_movement_heatmap:
    fig.write_html(os.path.join(output_folder, "movement_heatmap.html"))


In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np
from scipy.signal import find_peaks

avg_spikes = np.array(avg_spike_all_traces)

# Find global min and max values
global_min = np.min(avg_spikes)
global_max = np.max(avg_spikes)

# Define x-axis values
x_values = np.linspace(-range_value, range_value, avg_spikes.shape[1])

# Create subplots
fig = make_subplots(rows=1, cols=3, subplot_titles=("Positive Traces", "Insignificant Traces", "Negative Traces"))

# Define plot data in the desired order
plot_data = [
    (trace_id_sorted_sig_pos, 'Positive'),
    (trace_id_sorted_insig, 'Insignificant'),
    (trace_id_sorted_sig_neg, 'Negative')
]

for i, (trace_ids, title) in enumerate(plot_data, start=1):
    plot_spikes = avg_spikes[trace_ids]
    
    # Plot individual traces
    for spike in plot_spikes:
        fig.add_trace(go.Scatter(x=x_values, y=spike, mode='lines', line=dict(color='lightgrey', width=1), showlegend=False), row=1, col=i)
    
    # Plot mean trace
    mean_line = np.mean(plot_spikes, axis=0)
    fig.add_trace(go.Scatter(x=x_values, y=mean_line, mode='lines', line=dict(color='black', width=3), showlegend=False), row=1, col=i)

# Update layout with global y-axis range
fig.update_layout(
    title=f'Superimposed Traces and Mean for t value groups<br>{file_name}',
    width=1200,
    height=400,
    yaxis=dict(range=[global_min, global_max]),
    yaxis2=dict(range=[global_min, global_max]),
    yaxis3=dict(range=[global_min, global_max]),
    xaxis=dict(range=[-range_value, range_value], title='Time (frame)'),
    xaxis2=dict(range=[-range_value, range_value], title='Time (frame)'),
    xaxis3=dict(range=[-range_value, range_value], title='Time (frame)')
)

# Show plot
fig.show()

fig.write_html(os.path.join(output_folder, "pmth_average_t_groups.html"))

In [ ]:
from sortedcontainers import SortedDict

range_value = 104 # 104 from Cicada (int, how many steps (frames) to collect before and after the "stimulus")
frames_indices = np.arange(-1 * range_value, range_value + 1)
n_frames_psth = len(frames_indices)

def get_psth_values(data, epochs_frame, low_percentile = 25, high_percentile = 75):

    
    sum_activity_by_frame_dict = SortedDict()

    for epoch_index, epoch_frame in enumerate(epochs_frame):
        beg_frame = np.max((0, epoch_frame - range_value))
        end_frame = np.min((n_frames, epoch_frame + range_value + 1))

        # before the event
        sum_activity_before = np.sum(data[:, beg_frame:epoch_frame], axis=0) # axis 0 means cells 1, 2, 3,... are summed at each time frame.  
        frames_before = np.arange(-(epoch_frame - beg_frame), 0)

        for i, frame in enumerate(frames_before):
            if frame not in sum_activity_by_frame_dict:
                sum_activity_by_frame_dict[frame] = []
            if len(sum_activity_before) > i:
                sum_activity_by_frame_dict[frame].append(sum_activity_before[i])

        # after the event
        sum_activity_after = np.sum(data[:, epoch_frame:end_frame], axis=0)
        frames_after = np.arange(0, end_frame - epoch_frame)
        for i, frame in enumerate(frames_after):
            if frame not in sum_activity_by_frame_dict:
                sum_activity_by_frame_dict[frame] = []
            if len(sum_activity_after) > i:
                sum_activity_by_frame_dict[frame].append(sum_activity_after[i])


    fcts_to_apply = [np.nanmedian, lambda x: np.nanpercentile(x, low_percentile), lambda x: np.nanpercentile(x, high_percentile)]
    # fcts_to_apply = [np.nanmean, lambda x: np.nanpercentile(x, low_percentile), lambda x: np.nanpercentile(x, high_percentile)]


    psth_values = list()
    for fct_index in range(len(fcts_to_apply)):
        psth_values.append([])

    for frame in frames_indices:
        for fct_index, fct_to_apply in enumerate(fcts_to_apply): # fcts_to_apply contains 3 functions, 1) get median, 2) get low percentile (set at 25% in cicada_psth_analysis.py), and 3) get high percentile (75%).
            if frame not in sum_activity_by_frame_dict:
                psth_values[fct_index].append(0)
            else:
                sum_activity = sum_activity_by_frame_dict[frame]
                sum_activity = (fct_to_apply(sum_activity) / n_cells) # fct_to_apply is defined as:  np.nanpercentile(x, high_percentile)
                psth_values[fct_index].append(sum_activity)

    
    return psth_values

plot_psth_dff = get_psth_values(convert_to_dff(traces), movements_frame)


In [ ]:
def get_surr_psth_values(neuronal_data,epochs_frame):

    n_surrogates = 50 # 500 is default from Cicada 
    psth_frames_indices = frames_indices
    cell_indices_in_group = np.arange(n_cells)
    surrogate_method = "single cell roll"
    verbose = False

    surrogate_max_values = []
    # Will be use to take the median, low, high percentiles of the x surrogates at each time point
    surrogate_median_values = np.zeros((len(psth_frames_indices), n_surrogates), dtype=float)
    surrogate_low_perc_values = np.zeros((len(psth_frames_indices), n_surrogates), dtype=float)
    surrogate_high_perc_values = np.zeros((len(psth_frames_indices), n_surrogates), dtype=float)
    # Loop on the n surrogates
    for index_surrogate in range(n_surrogates):
        surrogate_neuronal_data = np.copy(neuronal_data[cell_indices_in_group])
        if surrogate_method == "single cell roll":
            if verbose and index_surrogate == 0:
                print(f"Proceed to independent rolling of all cells")
            for cell in np.arange(len(surrogate_neuronal_data)):
                roll_value = np.random.randint(1, surrogate_neuronal_data.shape[1])
                surrogate_neuronal_data[cell, :] = np.roll(surrogate_neuronal_data[cell, :], shift=roll_value)
        # if surrogate_method == "population roll":
        #     if verbose and index_surrogate == 0:
        #         print(f"Proceed to identical rolling of all cells")
        #     roll_value = np.random.randint(1, surrogate_neuronal_data.shape[1])
        #     surrogate_neuronal_data = np.roll(surrogate_neuronal_data, shift=roll_value, axis=1)

        surro_psth_values = get_psth_values(data=surrogate_neuronal_data,epochs_frame=epochs_frame,low_percentile=5,high_percentile=95)
        # Append the max values of the psth:
        surrogate_max_values.append([np.max(surro_psth_values[0])])

        # Create a matrix in which each column is the median value of a surrogate psth
        surrogate_median_values[:, index_surrogate] = surro_psth_values[0]
        surrogate_low_perc_values[:, index_surrogate] = surro_psth_values[1]
        surrogate_high_perc_values[:, index_surrogate] = surro_psth_values[2]
    # take the median at each time point for low prctile, median and high prctile of the surrogates
    surrogate_median_psth_values = np.nanmedian(surrogate_median_values, axis=1)
    surrogate_low_per_psth_values = np.nanmedian(surrogate_low_perc_values, axis=1)
    surrogate_high_per_psth_values = np.nanmedian(surrogate_high_perc_values, axis=1)
    surrogate_psth_values = [surrogate_median_psth_values, surrogate_low_per_psth_values,
                                surrogate_high_per_psth_values]
    
    return surrogate_psth_values

In [ ]:
plot_psth_dff = get_psth_values(traces_dff, movements_frame)
plot_psth_surr_dff = get_surr_psth_values(traces_dff, movements_frame)

In [ ]:
import plotly.graph_objects as go
from scipy.signal import find_peaks


psth_values = plot_psth_dff
psth_surr_values = plot_psth_surr_dff

psth_med_amplitude = round(np.max(psth_values[0])-np.median(psth_values[0]),2)

# Create a figure
fig = go.Figure()

first_frame = int((len(psth_values[0])-1)/-2)

# Add traces for each line
for i, values in enumerate(psth_values):
    if i == 0: # keep only middle line
        x_values = list(range(first_frame, first_frame + len(values)))
        fig.add_trace(go.Scatter(x=x_values, y=values, mode='lines', name=f'Line {i + 1}',showlegend=False))

# Add traces for each line in psth_surr_values
for i, values in enumerate(psth_surr_values):
    if i != 0: # removing middle line
        x_values = list(range(first_frame, first_frame + len(values)))
        color = 'gray'
        fig.add_trace(go.Scatter(x=x_values, y=values, mode='lines', name=f'Surrogate Line {i + 1}', line=dict(color=color), showlegend=False))


# Update layout
fig.update_layout(
    title=f'PMTH for all movements with surr. lines (95%, 5%)<br>{file_name}<br>Amplitude: {psth_med_amplitude}',
    xaxis=dict(title='Time (frame)', range=[first_frame, first_frame + len(psth_values[0]) - 1]),
    yaxis=dict(title='%DF/F (sum of fluo. signals)'),
    width=700,  # Set the width of the plot
    height=450  # Set the height of the plot
)

# Show the plot
fig.show()

fig.write_html(os.path.join(output_folder, "pmth_cicada.html"))

In [ ]:
### CREATE THREE SEPARATE PMTH FOR EACH MOVEMENT

plot_psth_dff_movement = []
plot_psth_surr_dff_movement = []

for movement in range(n_movement_types):
    plot_psth_dff_movement.append(get_psth_values(traces_dff, movements_frame_list[movement])) ### THIS PRODUCES 3 LISTS (movement) of 3 lists (median, low, high %)) 
    plot_psth_surr_dff_movement.append(get_surr_psth_values(traces_dff, movements_frame_list[movement]))

In [ ]:
### PLOT PSTH for individual movements

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np
from scipy.signal import find_peaks

# data
psth_values = plot_psth_dff_movement.copy()
psth_surr_values = plot_psth_surr_dff_movement.copy()

subtitles = []
for n in range(n_movement_types):
    psth_med_amplitude = (round(np.max(psth_values[n][0]) - np.median(psth_values[n][0]), 2))
    subtitles.append(f"{movement_types_all[n]}<br>Amplitude: {psth_med_amplitude}") ### DIFFERENT FROM Y MAX

# Define the number of subplots and their arrangement
rows = 1
cols = 3
fig = make_subplots(rows=rows, cols=cols, subplot_titles=subtitles)

first_frame = int((n_frames_psth-1)/-2)
x_values = list(range(first_frame, first_frame + n_frames_psth))

max_group = [np.max(sublist[0]) for sublist in psth_values]
min_group = [np.min(sublist[0]) for sublist in psth_values]
y_max, y_min = np.max(max_group), np.min(min_group)

# Define the number of subplots and their arrangement

# Add traces for each line in each subplot
for i in range(cols):
    
    fig.add_trace(go.Scatter(x=x_values, y=psth_values[i][0], mode='lines', name=f'Line {i + 1}', showlegend=False), row=1, col=i + 1)

    # Add traces for psth_surr_values
    for j, values in enumerate(psth_surr_values[i]):
        if j != 0:  # removing middle line
            color = 'gray'
            fig.add_trace(go.Scatter(x=x_values, y=psth_surr_values[i][j], mode='lines', name=f'Surrogate Line {j + 1}', line=dict(color=color), showlegend=False), row=1, col=i + 1)

# Update layout with shared y-axis range and x-axis values for each subplot
    fig.update_xaxes(title='Time (frame)', range=[x_values[0], x_values[-1]], row=1, col=i + 1)
    fig.update_yaxes(range=[y_min*1.4, y_max*1.1], row=1, col=i + 1)

# Update layout
fig.update_layout(
    title=f'PMTH for each movement with surr. lines (5% & 95%)  ||  {file_name}',
    title_x=0.5,
    yaxis=dict(title='%DF/F (sum of fluo. signals)'),
    width=1500,  # Set the total width of the plot
    height=450,  # Set the height of the plot
    grid=dict(rows=1, columns=3, pattern='independent'),  # Set independent grid pattern for subplots
    margin=dict(t=100),  # Add top margin for title
)

# Show the plot
fig.show()
fig.write_html(os.path.join(output_folder, "pmth_sep_movements.html"))

In [ ]:
from sortedcontainers import SortedDict

range_value = 104 # 104 from Cicada (int, how many steps (frames) to collect before and after the "stimulus")
frames_indices = np.arange(-1 * range_value, range_value + 1)
n_frames_psth = len(frames_indices)

def get_psth_sce(data, epochs_frame, low_percentile = 25, high_percentile = 75):

    
    sum_activity_by_frame_dict = SortedDict()

    for epoch_index, epoch_frame in enumerate(epochs_frame):
        beg_frame = np.max((0, epoch_frame - range_value))
        end_frame = np.min((n_frames, epoch_frame + range_value + 1))

        # before the event
        sum_activity_before = np.sum(data[:, beg_frame:epoch_frame], axis=0) # axis 0 means cells 1, 2, 3,... are summed at each time frame.  
        frames_before = np.arange(-(epoch_frame - beg_frame), 0)

        for i, frame in enumerate(frames_before):
            if frame not in sum_activity_by_frame_dict:
                sum_activity_by_frame_dict[frame] = []
            if len(sum_activity_before) > i:
                sum_activity_by_frame_dict[frame].append(sum_activity_before[i])

        # after the event
        sum_activity_after = np.sum(data[:, epoch_frame:end_frame], axis=0)
        frames_after = np.arange(0, end_frame - epoch_frame)
        for i, frame in enumerate(frames_after):
            if frame not in sum_activity_by_frame_dict:
                sum_activity_by_frame_dict[frame] = []
            if len(sum_activity_after) > i:
                sum_activity_by_frame_dict[frame].append(sum_activity_after[i])


    # fcts_to_apply = [np.nanmedian, lambda x: np.nanpercentile(x, low_percentile), lambda x: np.nanpercentile(x, high_percentile)]
    fcts_to_apply = [np.sum] 


    psth_values = list()
    for fct_index in range(len(fcts_to_apply)):
        psth_values.append([])

    for frame in frames_indices:
        for fct_index, fct_to_apply in enumerate(fcts_to_apply): # fcts_to_apply contains 3 functions, 1) get median, 2) get low percentile (set at 25% in cicada_psth_analysis.py), and 3) get high percentile (75%).
            if frame not in sum_activity_by_frame_dict:
                psth_values[fct_index].append(0)
            else:
                sum_activity = sum_activity_by_frame_dict[frame]
                sum_activity = (fct_to_apply(sum_activity)) # / n_cells) # fct_to_apply is defined as:  np.nanpercentile(x, high_percentile)
                psth_values[fct_index].append(sum_activity)

    
    return psth_values

plot_psth_sce = get_psth_sce(sce_trace, movements_frame)

In [ ]:
# # PLOT SCE in bins

# bin_in_sec = 1
# bin_in_frames = round(bin_in_sec * frames_per_second)

# psth_values = plot_psth_sce
# summed_values = []
# for values in psth_values:
#     summed_values.append([sum(values[i:i+bin_in_frames]) for i in range(0, len(values), bin_in_frames)])

# # Create a figure
# fig = go.Figure()

# first_frame = int((len(summed_values[0])-1)/-2)
# print(summed_values)
# # Add traces for each line
# for i, values in enumerate(summed_values):
#     x_values = list(range(first_frame, first_frame + len(values)))
#     fig.add_trace(go.Scatter(x=x_values, y=values, mode='lines', name=f'Line {i + 1}'))

# # Update layout
# fig.update_layout(
#     title=f'PMTH (of SCEs) for all movements with bins of {bin_in_sec} sec<br>{file_name}',
#     xaxis=dict(title='Time (seconds)', range=[first_frame, first_frame + len(summed_values[0]) - 1]),
#     yaxis=dict(title='SCE count'),
#     width=700,  # Set the width of the plot
#     height=450  # Set the height of the plot
# )

# # Show the plot
# fig.show()
# fig.write_html(os.path.join(output_folder, "pmth_sce.html"))

In [ ]:
import plotly.graph_objects as go
import os

# bin_in_sec = 1
# bin_in_frames = round(bin_in_sec * frames_per_second)
bin_in_frames = 1

psth_values = plot_psth_sce
summed_values = []
for values in psth_values:
    summed_values.append([sum(values[i:i+bin_in_frames]) for i in range(0, len(values), bin_in_frames)])

# Create a figure
fig = go.Figure()

# Adjust first_frame to be the starting frame of the x-axis
first_frame = -(len(psth_values[0])-1)/2

# Add traces for each line
for i, values in enumerate(summed_values):
    x_values = [first_frame + bin_in_frames * j for j in range(len(values))]
    fig.add_trace(go.Bar(x=x_values, y=values, width=bin_in_frames, name=f'Line {i + 1}'))

# Update layout
fig.update_layout(
    title=f'PMTH (of SCEs) for all movements with bins of 1 frame<br>{file_name}',
    xaxis=dict(title='Time (frame)', range=[first_frame, first_frame*-1]),
    yaxis=dict(title='SCE count'),
    barmode='overlay',  # Bars will be overlaid instead of stacked
    width=700,  # Set the width of the plot
    height=450  # Set the height of the plot
)

# Show the plot
fig.show()
fig.write_html(os.path.join(output_folder, "pmth_sce.html"))


In [ ]:
### Calculate MULTIPLE ave spikes from MULTIPLE traces
### FOCUS ON BIMODAL TRACES FROM INSIGNIFICANT GROUP

from scipy.stats import ttest_ind

traces_to_plot = traces_dff[trace_id_sorted_insig]
# traces_to_plot = traces_dff

# traces_to_plot = traces

range_value=104
lower_percent = 2.5 
upper_percent = 97.5

mini_post_analysis_length = 5 # from center

midpoint = range_value
pre_window_start = range_value-pre_analysis_length
mini_window_end = range_value+mini_post_analysis_length
post_window_end = range_value+post_analysis_length

avg_spike_all_traces = []
trace_length = len(traces[0])  # Assuming all traces have the same length
positive_trace_index = []
neutral_trace_index = []
negative_trace_index = []

maxs = []
mins = []
max_frames = []
min_frames = []
upper_thresholds = []
lower_thresholds = []

margins_to_threshold = []

pre_avgs = []
pre_stds = []
post_z_max = []
post_z_min = []

t_values = []
p_values = []

median_all_traces = []

for trace in traces_to_plot:
    spikes_per_trace = []
    
    median_all_traces.append(np.median(trace))

    upper_thresholds.append(np.percentile(traces_to_plot, upper_percent))
    lower_thresholds.append(np.percentile(traces_to_plot, lower_percent))

    for frame_index in movements_frame:
        # Calculate the start and end index for the segment
        beg_frame = max(frame_index - range_value, 0)  # Ensure beg_frame is not negative
        end_frame = min(frame_index + range_value + 1, trace_length)  # Ensure end_frame is within trace_length
        
        # Extract the segment from the time series and pad with zeros if necessary
        segment = trace[beg_frame:end_frame]

        if len(segment) < 2 * range_value + 1:
            pad_length = 2 * range_value + 1 - len(segment)
            segment = list(segment)
            segment.extend([0] * pad_length)
            segment = np.array(segment)          
        
        spikes_per_trace.append(segment)

    # print(len(segments))
    # Compute the average of the segments for the current trace
    avg_spike_per_trace = [sum(values) / len(spikes_per_trace) for values in zip(*spikes_per_trace)]
    avg_spike_all_traces.append(avg_spike_per_trace)

    ### here we're using a mini window after zero  
    pre_avg_spike_per_trace = avg_spike_per_trace[midpoint:mini_window_end] ### WHOLE WINDOW (NOT USING pre_analysis_length)
    post_avg_spike_per_trace = avg_spike_per_trace[mini_window_end:post_window_end]

    t_stat, p_value = ttest_ind(post_avg_spike_per_trace, pre_avg_spike_per_trace)
    t_values.append(t_stat)
    p_values.append(p_value)

    ### looking at negative and positive peaks only
    max_value = np.max(avg_spike_per_trace[midpoint:post_window_end])
    min_value = np.min(avg_spike_per_trace[midpoint:post_window_end])
    maxs.append(max_value)
    mins.append(min_value)

    max_index = np.argmax(avg_spike_per_trace[midpoint:post_window_end])
    min_index = np.argmin(avg_spike_per_trace[midpoint:post_window_end])
    max_frames.append(max_index)
    min_frames.append(min_index)

insig_mean_line = np.mean(plot_spikes, axis=0)



In [ ]:
######### SORTING BY REFOCUSING ON INSIGNIFICANT TRACES (not ARTEM's version because it compares the peak to the premovement average, whereas here we're comparing two peaks) 

threshold_value = 0.05 # for p values

# Get indices of values above and below 0.05
trace_id_sig = [i for i, p_val in enumerate(p_values) if p_val <= threshold_value]
trace_id_insig = [i for i, p_val in enumerate(p_values) if p_val > threshold_value]

# Sort only certain indices based on the condition
trace_id_sorted_sig = sorted(trace_id_sig, key=lambda idx: t_values[idx], reverse=True)
trace_id_sorted_insig = sorted(trace_id_insig, key=lambda idx: t_values[idx], reverse=True)

trace_id_sorted_sig_pos = []
trace_id_sorted_sig_neg = []

for i, idx in enumerate(trace_id_sorted_sig):
    if t_values[idx] >= 0:
        trace_id_sorted_sig_pos.append(idx)
        # print('pos ', i, ' ', t_values[idx])
    else:
        trace_id_sorted_sig_neg.append(idx)
        # print('neg ',i,' ', t_values[idx])

print(len(trace_id_sorted_sig_pos), ' ', len(trace_id_sorted_insig), ' ',len(trace_id_sorted_sig_neg) )
print(len(trace_id_sorted_sig_pos)+len(trace_id_sorted_sig_neg)+len(trace_id_sorted_insig))

In [ ]:
######### SORTING BY MIN MAX SLOPES (not ARTEM's version because it compares the peak to the premovement average, whereas here we're comparing two peaks) 

threshold_value = 0.05 # for p values

# Get indices of values above and below 0.05
# Initialize empty lists
trace_id_pos = []
trace_id_neg = []
trace_id_bi = []
trace_id_insig = []

# Single loop for enumeration and conditional logic
for i, (max_val, min_val, upper_threshold, lower_threshold, max_frame, min_frame) in enumerate(zip(maxs, mins,upper_thresholds,lower_thresholds, max_frames, min_frames)):
    if max_val >= upper_threshold and min_val >= lower_threshold:
        trace_id_pos.append(i)
    elif max_val <= upper_threshold and min_val <= lower_threshold:
        trace_id_neg.append(i)
    elif max_val >= upper_threshold and min_val < lower_threshold and min_frame > max_frame:
        trace_id_bi.append(i)
    elif max_val < upper_threshold and min_val >= lower_threshold:
        trace_id_insig.append(i)

n_trace_id_pos = len(trace_id_pos)
n_trace_id_neg = len(trace_id_neg)
n_trace_id_bi = len(trace_id_bi)
n_trace_id_insig =len(trace_id_insig)
print(f"check all traces captured: {n_trace_id_pos} {n_trace_id_neg} {n_trace_id_bi} {n_trace_id_insig}")

print(f"check all traces captured: {n_cells==n_trace_id_pos+n_trace_id_neg+n_trace_id_bi+n_trace_id_insig}")


# # Sort only certain indices based on the condition
# trace_id_sorted_sig = sorted(trace_id_sig, key=lambda idx: t_values[idx], reverse=True)
# trace_id_sorted_insig = sorted(trace_id_insig, key=lambda idx: t_values[idx], reverse=True)

# trace_id_sorted_sig_pos = []
# trace_id_sorted_sig_neg = []

# for i, idx in enumerate(trace_id_sorted_sig):
#     if t_values[idx] >= 0:
#         trace_id_sorted_sig_pos.append(idx)
#         # print('pos ', i, ' ', t_values[idx])
#     else:
#         trace_id_sorted_sig_neg.append(idx)
#         # print('neg ',i,' ', t_values[idx])

# print(len(trace_id_sorted_sig_pos), ' ', len(trace_id_sorted_insig), ' ',len(trace_id_sorted_sig_neg) )
# print(len(trace_id_sorted_sig_pos)+len(trace_id_sorted_sig_neg)+len(trace_id_sorted_insig))

In [ ]:
import plotly.graph_objs as go

export_movement_heatmap = True

sig_pos_count = len(trace_id_sorted_sig_pos)
sig_neg_count = len(trace_id_sorted_sig_neg)
insig_count = len(trace_id_sorted_insig)

total_count = sig_pos_count + sig_neg_count + insig_count

# plot_traces = np.array(avg_spike_all_traces)
plot_traces = stats.zscore(avg_spike_all_traces, axis=1)

order = trace_id_sorted_sig_pos + trace_id_sorted_insig + trace_id_sorted_sig_neg


order = order[::-1]
line1 = len(trace_id_sorted_sig_neg)
line2 = len(trace_id_sorted_sig_neg)+len(trace_id_sorted_insig)
first_frame = int((len(plot_traces[0])-1)/-2)
x_values = list(range(first_frame, first_frame + len(plot_traces)))


custom_colorscale = [
    [0.0, 'blue'],    # Start with white
    [0.5, 'white'],
    [1.0, 'red']     # End with black
]

layout = go.Layout(
    title=(f'Sorted PMTH heatmap of INSIGNIFICANT TRACES<br>{file_name} <br>'
           f'pos. t: {int(100*sig_pos_count/total_count)}% ({sig_pos_count}/{total_count})   |   '
           f'insig. t: {int(100*insig_count/total_count)}% ({insig_count}/{total_count})   |   '
           f'neg. t: {int(100*sig_neg_count/total_count)}% ({sig_neg_count}/{total_count})'),        
    xaxis=dict(title='Time (frame)'),
    yaxis=dict(title='Cells (middle group: p_val > 0.05)'),
    height=650,
    width=750,
    shapes=[
        dict(type="line", xref="paper", yref="y", x0=0, y0=line1, x1=1, y1=line1, line=dict(color="black", width=2)),
        dict(type="line", xref="paper", yref="y", x0=0, y0=line2, x1=1, y1=line2, line=dict(color="black", width=2))
    ]
)

fig = go.Figure(layout=layout)


# Add the heatmap trace with zmin and zmax set to -2 and 2
fig.add_trace(go.Heatmap(
    x=x_values, 
    z=plot_traces[order], 
    colorscale=custom_colorscale,
    zmin=-1.96,
    zmax=1.96
))
fig.show()

if export_movement_heatmap:
    fig.write_html(os.path.join(output_folder, "movement_heatmap_insig_only.html"))


In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np
from scipy.signal import find_peaks

avg_spikes = np.array(avg_spike_all_traces)

# Find global min and max values
global_min = np.min(avg_spikes)
global_max = np.max(avg_spikes)

# Define x-axis values
x_values = np.linspace(-range_value, range_value, avg_spikes.shape[1])

# Create subplots
fig = make_subplots(rows=1, cols=3, subplot_titles=("Positive Traces", "Insignificant Traces", "Negative Traces"))

# Define plot data in the desired order
plot_data = [
    (trace_id_sorted_sig_pos, 'Positive'),
    (trace_id_sorted_insig, 'Insignificant'),
    (trace_id_sorted_sig_neg, 'Negative')
]

for i, (trace_ids, title) in enumerate(plot_data, start=1):
    plot_spikes = avg_spikes[trace_ids]
    
    # Plot individual traces
    for spike in plot_spikes:
        fig.add_trace(go.Scatter(x=x_values, y=spike, mode='lines', line=dict(color='lightgrey', width=1), showlegend=False), row=1, col=i)
    
    # Plot mean trace
    mean_line = np.mean(plot_spikes, axis=0)
    fig.add_trace(go.Scatter(x=x_values, y=mean_line, mode='lines', line=dict(color='black', width=3), showlegend=False), row=1, col=i)

# Update layout with global y-axis range
fig.update_layout(
    title=f'Superimposed Traces and Mean for t value groups of INSIGNIFICANT TRACES<br>{file_name}',
    width=1200,
    height=400,
    yaxis=dict(range=[global_min, global_max]),
    yaxis2=dict(range=[global_min, global_max]),
    yaxis3=dict(range=[global_min, global_max]),
    xaxis=dict(range=[-range_value, range_value], title='Time (frame)'),
    xaxis2=dict(range=[-range_value, range_value], title='Time (frame)'),
    xaxis3=dict(range=[-range_value, range_value], title='Time (frame)')
)

# Show plot
fig.show()

fig.write_html(os.path.join(output_folder, "pmth_average_t_groups_insig_only.html"))

In [ ]:
import numpy as np
import plotly.graph_objects as go
import random
from scipy.signal import savgol_filter

# Sample list of time series (each time series is a numpy array)
# Pick a random time series
# ts = random.choice(avg_spike_all_traces)

insig_mean_line = np.mean(plot_spikes, axis=0)
# plt.plot(insig_mean_line)

ts = insig_mean_line

# trace_id = avg_spike_all_traces.index(ts)
trace_id = 0

time = np.arange(len(ts))

# Define the time frame
start, end = range_value, range_value + post_analysis_length

# Find the max and min values within the specified time frame
max_val_idx = np.argmax(ts[start:end + 1]) + start
min_val_idx = np.argmin(ts[start:end + 1]) + start

# Apply Savitzky-Golay filter for smoothing
smoothed_ts = savgol_filter(ts, window_length=20, polyorder=2) #11
# smoothed_ts = savgol_filter(ts, window_length=7, polyorder=3) ### from JC


# Compute the first derivative
first_derivative = np.diff(smoothed_ts, prepend=ts[0])
# first_derivative = np.diff(ts, prepend=ts[0])

# Calculate the median of the original time series
median_value = median_all_traces[trace_id]

# Create the plot
fig = go.Figure()

# Original time series
fig.add_trace(go.Scatter(x=time, y=ts, mode='lines', line=dict(color='gray'), name=f'Time Series (Trace ID: {trace_id})'))

# Smoothed time series
fig.add_trace(go.Scatter(x=time, y=smoothed_ts, mode='lines', line=dict(color='black'), name='Smoothed Time Series'))

# First derivative
fig.add_trace(go.Scatter(x=time, y=first_derivative, mode='lines', line=dict(color='red'), name='First Derivative of smoothed'))

# Max and min values within the specified time frame
fig.add_trace(go.Scatter(x=[time[max_val_idx]], y=[ts[max_val_idx]], mode='markers', marker=dict(color='green', size=10), name='Max Value'))
fig.add_trace(go.Scatter(x=[time[min_val_idx]], y=[ts[min_val_idx]], mode='markers', marker=dict(color='blue', size=10), name='Min Value'))

# Highlight the time frame
fig.add_vline(x=start, line=dict(color='grey', dash='dash'))
fig.add_vline(x=end, line=dict(color='grey', dash='dash'))

# Add the horizontal median line
fig.add_hline(y=median_value, line=dict(color='purple', dash='dash'), name='Median Value')

# Add labels and title
fig.update_layout(title='Average of all traces + first derivative',
                  xaxis_title='Time',
                  yaxis_title='Value',
                  legend_title='Legend',
                  width=1000,  # Set the width of the plot
                  height=600)  # Set the height of the plot

# Display the plot
fig.show()

fig.write_html(os.path.join(output_folder, "pmth_average_insig_derivative.html"))
# 108about:blank#blocked

In [ ]:
# def save_variables_to_txt(filename, **kwargs):
#     with open(filename, 'w') as f:
#         for key, value in kwargs.items():
#             f.write(f"{key}: {value}\n")

# # Example usage
# save_variables_to_txt(os.path.join(suite2p_folder, "export_data"), 
#                       sce_count=n_sce, 
#                       sce_per_minute=sce_per_min,
#                       sce_threshold=sce_n_cells_threshold,
#                       spikes_count=n_spikes,
#                       spikes_per_min=spikes_per_min, 
#                       spikes_per_min_per_cell = spikes_per_min_per_cell,
#                       movement_count=movement_count,
#                       movement_per_min=movement_per_min)



In [ ]:
from openpyxl import load_workbook

# Load your workbook and select the sheet
workbook = load_workbook(main_excel_path)
sheet = workbook.active

# Define the column names you want to use starting from column 3
column_names = [
    'sce_n_cells_threshold', 'sce_count', 'sce_per_sec', 'spikes_count', 'spikes_per_sec', 
    'spikes_per_sec_per_cell', 'movement_count', 'movement_per_min', 'sig_pos_count', 
    'insig_count', 'sig_neg_count'
]

# Create the column indices starting from column 3
column_indices = {name: i+4 for i, name in enumerate(column_names)}

# Write the column names into the header row starting from column 3
for name, col in column_indices.items():
    sheet.cell(row=1, column=col, value=name)

# Insert the values into the specified cells
sheet.cell(row=excel_row_id + 2, column=column_indices['sce_n_cells_threshold'], value=sce_n_cells_threshold)
sheet.cell(row=excel_row_id + 2, column=column_indices['sce_count'], value=n_sce)
sheet.cell(row=excel_row_id + 2, column=column_indices['sce_per_sec'], value=sce_per_sec)
sheet.cell(row=excel_row_id + 2, column=column_indices['spikes_count'], value=n_spikes)
sheet.cell(row=excel_row_id + 2, column=column_indices['spikes_per_sec'], value=spikes_per_sec)
sheet.cell(row=excel_row_id + 2, column=column_indices['spikes_per_sec_per_cell'], value=spikes_per_sec_per_cell)
sheet.cell(row=excel_row_id + 2, column=column_indices['movement_count'], value=movement_count)
sheet.cell(row=excel_row_id + 2, column=column_indices['movement_per_min'], value=movement_per_min)

sheet.cell(row=excel_row_id + 2, column=column_indices['sig_pos_count'], value=len(trace_id_sorted_sig_pos))
sheet.cell(row=excel_row_id + 2, column=column_indices['insig_count'], value=len(trace_id_sorted_insig))
sheet.cell(row=excel_row_id + 2, column=column_indices['sig_neg_count'], value=len(trace_id_sorted_sig_neg))

# Save the workbook
workbook.save(main_excel_path)
